In [ ]:
# The script's purpose is to apply the NLP analysis to 100 items from MongoDB and output JSON file

In [101]:
# Importing data for the task

import pymongo

con = pymongo.MongoClient(host ='127.0.0.1:27018', username = 'root' ,password = 'abc123' )
data = con['web_content']['articles'].find({},{'_id':0}) # without id

# all data
# data = [x for x in con['web_content']['articles'].find({},{'_id':0})]

# example of data item
data[1]

{'title': 'Northumbria: Can you help us trace wanted man Matthew Attenborough?',
 'post date': '13/05/2022',
 'link': 'https://crimestoppers-uk.org/campaigns-media/news/2022/may/northumbria-can-you-help-us-trace-wanted-man-matthew-attenborough',
 'content': 'We are offering a reward of up to £1,000 for anonymous information that leads to the arrest of 41-year-old Matthew Attenborough who is wanted in connection with a spate of fraud by false representation offences.\n\r\nA number of police forces across the country have been working together in a bid to locate Attenborough, who also goes by the name Rauiri O’Chonghaile, and is actively evading arrest.'}

In [102]:
# Transforming the data for further analysis
# Creating a list with 100 entries of text

def makeList(data):
    output = []
    for x in range(100):
        output.append(data[x].get("title") + " " + data[x].get("content"))
    return output


list_data = makeList(data)
print(list_data)

['Newcastle: Attack outside nightclub sees our charity offer reward & appeal for info Following an assault on a young man in Newcastle who was left with serious injuries, Crimestoppers is offering a reward and urging anyone with information to contact us anonymously.At just after midnight on 12 February 2022, the victim, a 20-year-old, was queuing to enter Flares nightclub in the city centre on Moseley Street.Another man was waiting behind him and accused him of queue jumping. Shortly after, the victim was punched in the head numerous times causing him to fall to the ground. Once there, he was repeatedly kicked.The perpetrator fled, but returned a short time later, but was prevented from continuing the assault by a man in a red jacket.The victim was rushed to hospital and sustained serious facial and ankle injuries.***Crimestoppers is supporting the investigation by offering a reward of up\xa0to £1,000 for information the charity exclusively receives - via our website or by phoning 080

In [103]:
# Defining an empty output object

output_dic = {'Suspect': [],
              'Location': [],
              'Date': [],
              'Offences': [],
              'Drugs': [],
              'Injury': [],
              'Death': []}

print(output_dic)

{'Suspect': [], 'Location': [], 'Date': [], 'Offences': [], 'Drugs': [], 'Injury': [], 'Death': []}


In [104]:
# Defining lookup dictionaries for further use

offences = ['robbery', 'stabbing', 'assault', 'affray', 'dangerous driving', 'possession with intent to supply', 'theft', 'money laundering', 'conspiracy', 'distribution', 'drugs', 'class b', 'class b', 'possession of a weapon', 'stolen goods', 'murder', 'burglary', 'fraud', 'rape', 'sexual assault', 'stealing', "Hit-and-run"]

drugs = ['cocaine', 'cannabis', 'heroin', 'MDMA', 'methylphenidate', 'diazepam', 'LSD', 'amphetamine', 'diamorphine']

regions = ['Avon & Somerset', 'Bedfordshire', 'Berkshire', 'Buckinghamshire', 'Cambridgeshire', 'Cheshire', 'Cleveland', 'Cumbria', 'Derbyshire', 'Devon & Cornwall', 'Dorset', 'Durham', 'Dyfed-Powys', 'Essex', 'Gloucestershire', 'Greater Manchester', 'Guernsey', 'Gwent', 'Hampshire & IOW', 'Hertfordshire', 'Humberside', 'Isle of Man', 'Jersey', 'Kent', 'Lancashire', 'Leicestershire', 'Lincolnshire', 'London', 'Merseyside', 'Nationwide', 'Norfolk', 'North Wales', 'North Yorkshire', 'Northamptonshire', 'Northern Ireland', 'Northumbria', 'Nottinghamshire', 'Oxfordshire', 'Scotland', 'South Wales', 'South Yorkshire', 'Staffordshire', 'Suffolk', 'Surrey', 'Sussex', 'Warwickshire', 'West Mercia', 'West Midlands', 'West Yorkshire', 'Wiltshire']

In [105]:
# Obtaining "Date" value

# Additional dataset for dates

def makeListOfDates(data):
    output = []
    for x in range(100):
        output.append(data[x].get("post date"))
    return output


list_data_dates = makeListOfDates(data)
# print(list_data_dates)

output_dic.update({"Date": list_data_dates})

print(output_dic)

{'Suspect': [], 'Location': [], 'Date': ['25/05/2022', '13/05/2022', '29/04/2022', '19/04/2022', '22/03/2022', '17/03/2022', '16/03/2022', '15/02/2022', '03/02/2022', '15/12/2021', '14/12/2021', '25/11/2021', '04/11/2021', '28/09/2021', '13/09/2021', '18/06/2021', '11/05/2021', '08/03/2021', '16/02/2021', '10/02/2021', '08/02/2021', '04/02/2021', '02/02/2021', '27/01/2021', '17/12/2020', '23/11/2020', '20/10/2020', '01/10/2020', '16/09/2020', '16/09/2020', '26/08/2020', '10/08/2020', '07/08/2020', '21/05/2020', '12/02/2020', '11/02/2020', '04/10/2019', '11/07/2019', '25/05/2022', '13/05/2022', '29/04/2022', '19/04/2022', '22/03/2022', '17/03/2022', '16/03/2022', '15/02/2022', '03/02/2022', '15/12/2021', '14/12/2021', '25/11/2021', '04/11/2021', '28/09/2021', '13/09/2021', '18/06/2021', '11/05/2021', '08/03/2021', '16/02/2021', '10/02/2021', '08/02/2021', '04/02/2021', '02/02/2021', '27/01/2021', '17/12/2020', '23/11/2020', '20/10/2020', '01/10/2020', '16/09/2020', '16/09/2020', '26/08/

In [106]:
# Obtaining 'Suspect' value

# Performing Named Entity Recognition to extract the name of the suspect from the text

import spacy
from pprint import pprint

nlp = spacy.load("en_core_web_sm")
doc = nlp(list_data[6])
result = [(X.text, X.label_) for X in doc.ents]

# print(result)

def onlyPerson(x):
    if x[1] == 'PERSON':
        return True
    else:
        return False
    
# print(list(filter(onlyPerson, result))[0][0])

list_of_suspects = []

for x in range(100):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(list_data[x])
    result = [(X.text, X.label_) for X in doc.ents]
    # print(result)
    
    def onlyPerson(y):
        if y[1] == 'PERSON':
            return True
        else:
            return False
    if list(filter(onlyPerson, result)) == []:
        list_of_suspects.append("UNKNOWN")
    else:
        list_of_suspects.append(list(filter(onlyPerson, result))[0][0])

# print(list_of_suspects)

output_dic.update({"Suspect": list_of_suspects})

print(output_dic)


{'Suspect': ['UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia Grainger', 'Andrew Jackson', 'Surrey', 'Hussain', 'UNKNOWN', 'UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia 

In [107]:
# Defining a function that will do the preprocessing for further analysis

import os
import nltk
from nltk.tokenize import word_tokenize

list_data_tagged = []

for x in range(100):
    item_tokenized = word_tokenize(list_data[x])
    item_tagged = nltk.pos_tag(item_tokenized)
    list_data_tagged.append(item_tagged)

print(len(list_data_tagged))

100


In [108]:
# Obtaining 'Location' value

def onlyRegion(x):
    if x[0] in regions:
        return True
    else:
        return False

list_of_regions = []


# print(list_data_tagged[0])
# print( list(filter(onlyRegion, list_data_tagged[10])) )
# []
# [('Northumbria', 'NNS')]
# [('London', 'NNP'), ('Surrey', 'NNP'), ('Surrey', 'NNP')]

for i in range(100):
    if list(filter(onlyRegion, list_data_tagged[i])) == []:
        list_of_regions.append("UNKNOWN")
    else:
        list_of_regions.append(list(filter(onlyRegion, list_data_tagged[i]))[0][0])

# print(list_of_regions)


output_dic.update({"Location": list_of_regions})

print(output_dic)

{'Suspect': ['UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia Grainger', 'Andrew Jackson', 'Surrey', 'Hussain', 'UNKNOWN', 'UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia 

In [109]:
# Obtaining 'Offences' value

# print(list_data_tagged[2])

def extractOffences(text_tagged):
    def onlyOffences(x):
        if x[0] in offences:
            return True
        else:
            return False
    return [x[0] for x in list(filter(onlyOffences, text_tagged))]   

Offences = extractOffences(list_data_tagged[3])

# ['murder', 'murder', 'murder', 'murder']
# []
# ['assault', 'murder']

#print(Offences)

list_of_offences = []

for i in range(100):
    if extractOffences(list_data_tagged[i]) == []:
        list_of_offences.append("UNKNOWN")
    else:
        list_of_offences.append(list(set(extractOffences(list_data_tagged[i]))))

# print(list_of_offences)

output_dic.update({"Offences": list_of_offences})

print(output_dic)

{'Suspect': ['UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia Grainger', 'Andrew Jackson', 'Surrey', 'Hussain', 'UNKNOWN', 'UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia 

In [110]:
# Obtaining 'Drugs' value

def extractDrugs(text_tagged):
    def onlyDrugs(x):
        if x[0] in drugs:
            return True
        else:
            return False
    return [x[0] for x in list(filter(onlyDrugs, text_tagged))]   

list_of_drugs = []

for i in range(100):
    if extractDrugs(list_data_tagged[i]) == []:
        list_of_drugs.append("UNKNOWN")
    else:
        list_of_drugs.append(list(set(extractDrugs(list_data_tagged[i]))))
        
#print(list_of_drugs)

output_dic.update({"Drugs": list_of_drugs})

print(output_dic)

# Can it be that in this data set there were no drugs at all?

# # Positive testing: let's intoduce keywords 'cocaine', 'cannabis', 'heroin' to the input above.

# def makeList_with_drugs(data):
#     output = []
#     for x in range(100):
#         output.append(data[x].get("title") + " cocaine " + data[x].get("content"))
#     return output

# list_data_with_drugs = makeList_with_drugs(data)

# #print(list_data_with_drugs)

# list_data_tagged_with_drugs = []

# for x in range(100):
#     item_tokenized = word_tokenize(list_data_with_drugs[x])
#     item_tagged = nltk.pos_tag(item_tokenized)
#     list_data_tagged_with_drugs.append(item_tagged)

# list_of_drugs_with_drugs = []   
    
# for i in range(100):
#     if extractDrugs(list_data_tagged_with_drugs[i]) == []:
#         list_of_drugs_with_drugs.append("UNKNOWN")
#     else:
#         list_of_drugs_with_drugs.append(list(set(extractDrugs(list_data_tagged_with_drugs[i]))))
        
# print(list_of_drugs_with_drugs)
# # works!, so yes we just happened to have a data set without drugs

{'Suspect': ['UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia Grainger', 'Andrew Jackson', 'Surrey', 'Hussain', 'UNKNOWN', 'UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia 

In [121]:
# Obtaining 'Injury' value

list_of_injury = []

def extractInjury(Offences):
    Injury_yes = ['stabbing', 'assault']
    if any(x in Offences for x in Injury_yes):
        return True
    else:
        return False
    
for i in range(100):
    list_of_injury.append(extractInjury(list_of_offences[i]))
    
output_dic.update({'Injury': list_of_injury})

print(output_dic)

{'Suspect': ['UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia Grainger', 'Andrew Jackson', 'Surrey', 'Hussain', 'UNKNOWN', 'UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia 

In [122]:
# Obtaining 'Death' value

list_of_death = []

def extractDeath(Offences):
    Death_yes = ['murder']
    if any(x in Offences for x in Death_yes):
        return True
    else:
        return False
    
for i in range(100):
    list_of_death.append(extractDeath(list_of_offences[i]))
    
output_dic.update({'Death': list_of_death})

print(output_dic)

{'Suspect': ['UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia Grainger', 'Andrew Jackson', 'Surrey', 'Hussain', 'UNKNOWN', 'UNKNOWN', 'Matthew Attenborough', 'Darren Whitehouse', "Ola Raji's", 'Tom Kirwan’s', 'Mohammed Ali Ege', 'Elidon Elezi', 'Lee Boxell', 'North Devon', 'UNKNOWN', 'Hunt', 'UNKNOWN', 'Joseph Sharpe', 'Openreach', 'UNKNOWN', 'Kieran Williams', 'UNKNOWN', 'UNKNOWN', 'Border Collie', 'Surrey', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Donna', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'Billy McCullagh’s', "Patricia Grainger's", 'UNKNOWN', 'Patricia 